In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions
from debiaswe.data import load_gender_seed

import torch

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [3]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')
print(E)
words = E.words
print("Words:", len(words))

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Words: 26423
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [4]:
# gender direction
v_gender = E.diff('she', 'he')

# Uncomment below for direction based on multiple definitional pairs
# with open('./data/definitional_pairs.json', "r") as f:
#     defs = json.load(f)
# v_gender = we.doPCA(defs, E).components_[0]

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [5]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender, thresh=1)

# for (a,b,c) in a_gender:
#     print(a+"-"+b)
we.viz(a_gender)

Computing neighbors
Mean: 10.219732808538016
Median: 7.0
(135018, 300)
   0                          she | he                           1.00
   1                      herself | himself                      0.92
   2                          her | his                          0.90
   3                        woman | man                          0.75
   4                     daughter | son                          0.67
   5                businesswoman | businessman                  0.65
   6                         girl | boy                          0.65
   7                      actress | actor                        0.65
   8                   chairwoman | chairman                     0.63
   9                      heroine | hero                         0.62
  10                       mother | father                       0.60
  11                  spokeswoman | spokesman                    0.59
  12                       sister | brother                      0.59
  13               

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [6]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.23798442, 'maestro'),
  (-0.21665451, 'statesman'),
  (-0.2075867, 'skipper'),
  (-0.20267203, 'protege'),
  (-0.20206761, 'businessman'),
  (-0.19492391, 'sportsman'),
  (-0.18836352, 'philosopher'),
  (-0.1807366, 'marksman'),
  (-0.17289859, 'captain'),
  (-0.16785556, 'architect'),
  (-0.16702037, 'financier'),
  (-0.1631364, 'warrior'),
  (-0.15280864, 'major_leaguer'),
  (-0.15001445, 'trumpeter'),
  (-0.14718868, 'broadcaster'),
  (-0.14637241, 'magician'),
  (-0.14401694, 'fighter_pilot'),
  (-0.13782285, 'boss'),
  (-0.13718201, 'industrialist'),
  (-0.13684887, 'pundit')],
 [(0.19714224, 'interior_designer'),
  (0.20833439, 'housekeeper'),
  (0.21560377, 'stylist'),
  (0.22363171, 'bookkeeper'),
  (0.23776126, 'maid'),
  (0.24125953, 'nun'),
  (0.24782577, 'nanny'),
  (0.24929333, 'hairdresser'),
  (0.24946159, 'paralegal'),
  (0.25276467, 'ballerina'),
  (0.2571882, 'socialite'),
  (0.26647124, 'librarian'),
  (0.27317625, 'receptionist'),
  (0.2754029, 'waitress'),
  

## Find soft debias transform using SGD
This code is largely based on code from https://github.com/TManzini/DebiasMulticlassWordEmbedding.

In [7]:
from debiaswe.debias import soft_debias

In [8]:
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)

In [ ]:
soft_debias(E, gender_specific_words, defs)

Loss @ Epoch #0: 870579.0625
Loss @ Epoch #10: 593107.375
Loss @ Epoch #20: 493500.875
Loss @ Epoch #30: 421578.0625
Loss @ Epoch #40: 365126.0
Loss @ Epoch #50: 319241.96875
Loss @ Epoch #60: 281073.71875
Loss @ Epoch #70: 248779.09375
Loss @ Epoch #80: 221095.15625
Loss @ Epoch #90: 197132.765625
Loss @ Epoch #100: 176237.0
Loss @ Epoch #110: 157912.171875
Loss @ Epoch #120: 141769.125
Loss @ Epoch #130: 127493.609375
Loss @ Epoch #140: 114828.3828125
Loss @ Epoch #150: 103559.0703125
Loss @ Epoch #160: 93505.9921875
Loss @ Epoch #170: 84516.9921875
Loss @ Epoch #180: 76462.21875
Loss @ Epoch #190: 69230.8203125
Loss @ Epoch #200: 62727.5
Loss @ Epoch #210: 56869.50390625
Loss @ Epoch #220: 51585.38671875
Loss @ Epoch #230: 46812.73828125
Loss @ Epoch #240: 42496.90234375
Loss @ Epoch #250: 38590.0546875
Loss @ Epoch #260: 35050.0234375
Loss @ Epoch #270: 31839.5546875
Loss @ Epoch #280: 28925.6875
Loss @ Epoch #290: 26279.12890625
Loss @ Epoch #300: 23873.787109375
Loss @ Epoch #310

In [ ]:
# save soft debiased embeddings
E.save('./embeddings/w2v_gnews_small_soft_debiased.txt')

In [ ]:
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)
we.viz(a_gender_debiased)

In [ ]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]